In [20]:
import numpy as np
import pandas as pd
from pathlib import Path
print(Path.cwd())
files = Path.cwd().joinpath("data")
songs = files.joinpath("songs")
metadata = files/"birdsong_metadata.csv"

df = pd.read_csv(metadata)
df.head(20)

df.nunique(0)
# df.info()
# # Construct file path by concatenating fold and file name
# df['relative_path'] = '/fold' + df['fold'].astype(str) + '/' + df['slice_file_name'].astype(str)
# 
# # Take relevant columns
# df = df[['relative_path', 'classID']]
# df.head()

C:\Users\bpk_e\PycharmProjects\pythonProject1


file_id                   264
genus                      66
species                    85
english_cname              88
who_provided_recording     68
country                    18
latitude                  195
longitute                 198
type                       54
license                     4
dtype: int64